# TSA Chapter 6: Case Study - Residual Diagnostics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch6/TSA_ch6_case_diagnostics/TSA_ch6_case_diagnostics.ipynb)

VAR residual diagnostics for interest rate model.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas-datareader -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, grangercausalitytests, acf
from scipy import stats
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Download interest rate data for cointegration case study
try:
    gs10 = web.DataReader('GS10', 'fred', '1990-01-01', '2024-01-01').resample('MS').last().dropna()
    gs2 = web.DataReader('GS2', 'fred', '1990-01-01', '2024-01-01').resample('MS').last().dropna()
    rates = pd.DataFrame({'10Y': gs10.iloc[:,0], '2Y': gs2.iloc[:,0]}).dropna()
except:
    np.random.seed(42); n = 408; dates = pd.date_range('1990-01-01', periods=n, freq='MS')
    rw = np.cumsum(np.random.normal(0, 0.1, n)) + 5
    rates = pd.DataFrame({'10Y': rw + np.random.normal(0, 0.2, n), '2Y': rw - 0.5 + np.random.normal(0, 0.15, n)}, index=dates)
print(f'Rates data: {len(rates)} obs')

diff_rates = rates.diff().dropna()
model = VAR(diff_rates); res = model.fit(2)
resid = res.resid

fig, axes = plt.subplots(2, 3, figsize=(14, 7))
for row, (col, color) in enumerate(zip(diff_rates.columns, [BLUE, RED])):
    r = resid[col].values
    # Residual time series
    axes[row, 0].plot(r, color=color, lw=0.5)
    axes[row, 0].axhline(0, color=GRAY, lw=0.5)
    axes[row, 0].set_title(f'Residuals: $\\Delta${col}', fontweight='bold')
    axes[row, 0].set_xlabel('Time')
    # ACF
    nlags = 20; acf_vals = acf(r, nlags=nlags)
    ci = 1.96 / np.sqrt(len(r))
    axes[row, 1].bar(range(nlags+1), acf_vals, color=color, alpha=0.7, width=0.6)
    axes[row, 1].axhline(ci, color=RED, ls='--', lw=0.8, label='95% CI')
    axes[row, 1].axhline(-ci, color=RED, ls='--', lw=0.8)
    axes[row, 1].set_title(f'ACF: $\\Delta${col} Residuals', fontweight='bold')
    axes[row, 1].set_xlabel('Lag')
    axes[row, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=1, frameon=False)
    # Q-Q plot
    (osm, osr), (slope, intercept, _) = stats.probplot(r, dist='norm')
    axes[row, 2].scatter(osm, osr, color=color, s=8, alpha=0.5)
    axes[row, 2].plot(osm, slope * np.array(osm) + intercept, color=RED, lw=1.5, label='Normal line')
    axes[row, 2].set_title(f'Q-Q: $\\Delta${col} Residuals', fontweight='bold')
    axes[row, 2].set_xlabel('Theoretical Quantiles'); axes[row, 2].set_ylabel('Sample Quantiles')
    axes[row, 2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=1, frameon=False)

plt.tight_layout(); save_chart(fig, 'ch6_case_diagnostics'); plt.show()